In [30]:
# !pip install --upgrade opencv_jupyter_ui

In [1]:
!python --version

Python 3.8.13


In [2]:
from myreader import MyReader
import cv2
from opencv_jupyter_ui import cv2_imshow, cv2_destroyAllWindows

In [10]:
reader=MyReader('/raid/home/labusermodaresi/realSense/2022-04-14-first-analysis/experiments/rec-2022-05-02-downstair/ali/running/02-05-2022-17-06-49/cam1')

In [58]:
import mediapipe as mp
import numpy as np
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
BG_COLOR = (192, 192, 192)#gray

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    result = model.process(image)                 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    mp_drawing.draw_landmarks(
        image,
        result.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    annotated_image=None
    if result.pose_landmarks:
        annotated_image=image.copy()
        condition = np.stack((result.segmentation_mask,) * 3, axis=-1) > 0.1
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR
        annotated_image = np.where(condition, annotated_image, bg_image)
        annotated_image=trim(annotated_image,condition)
    return image, result,annotated_image
def trim(arr, mask):
    bounding_box = tuple(
        slice(np.min(indexes), np.max(indexes) + 1)
        for indexes in np.where(mask))
    return arr[bounding_box]
try:
 reader.seek(70)
 with mp_pose.Pose(static_image_mode=0,
    #                       model_complexity=1, 
                          enable_segmentation=True,
                          min_detection_confidence=0.3) as pose:
    while reader.next():
        color=reader.current_color
        depth=reader.current_depth
        depth_color=reader.colorize_current_depth()

        color2=color[500:800,900:1200,:]
        img,res,an=mediapipe_detection(color2,pose)
        cv2_imshow('skeleton',img,width='100%',colorspace='rgb')
        cv2_imshow('color - depth',cv2.hconcat([color,depth_color]),width='100%',colorspace='bgr')
        cv2_imshow('segmented',an,width='100%',colorspace='bgr')
    #     cv2_imshow('depth',,width=300)
        print(reader.current_frame,end='\r')
        # break
except KeyboardInterrupt:pass
    

In [57]:
cv2.rectangle(color,(980,485),(1000,955),(0,0,255),4)
cv2_imshow('color - depth2',cv2.hconcat([color,depth_color]),width='100%')

In [53]:
reader.get_distance(980,485,980,955)

3.4948600833065266

In [55]:
reader.get_depth(980,485)

10.57800050242804